In [1]:
import dxpy
import subprocess

# Automatically discover dispensed dataset ID and load the dataset 
dispensed_dataset_id = dxpy.find_one_data_object(typename='Dataset', name='app*.dataset', folder='/', name_mode='glob')['id']

# Get project ID
project_id = dxpy.find_one_project()["id"]
dataset = (':').join([project_id, dispensed_dataset_id])

cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)


0

In [2]:
import os
import glob
import pandas as pd

path = os.getcwd()

data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_0", "name"].values
)

field_names_str = [f"olink_instance_0.{f}" for f in field_names]


/tmp/ipykernel_104/336538462.py:8: DtypeWarning: Columns (4,7,8,9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict_df = pd.read_csv(data_dict_csv)


In [3]:
import pyspark

config = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max', '128'),('spark.sql.execution.arrow.pyspark.enabled','true')])  
sc = pyspark.SparkContext(conf=config)
spark = pyspark.sql.SparkSession(sc)

In [9]:
for i in range(6):
    field_names_protein = ",".join(field_names_str[i*500:min((i+1)*500, len(field_names_str))])
    cmd = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names_protein,
    "--delimiter",
    ",",
    "--output",
    f"olink_{i}.sql",
    "--sql",
    ]
    subprocess.check_call(cmd)

    with open(f'olink_{i}.sql', 'r') as file:
        retrieve_sql = file.read()

    temp_df = spark.sql(retrieve_sql.strip(";"))
    
    pdf = temp_df.toPandas()
    pdf.to_csv(f'olink_{i}.csv', index=False)

CalledProcessError: Command '['dx', 'extract_dataset', 'project-J36kyb8JB5QV410jF8P3GjZ4:record-J36qY8jJzx44Bv6754b5zQfX', '--fields', '', '--delimiter', ',', '--output', 'olink_6.sql', '--sql']' returned non-zero exit status 1.

0
1
2
3
4
5
